#### ISSUES:

* texts are repeating (solved)
* issues with some glosses (not lists)
* what to do about numeral / noun compounds? сом-зкьЫ-зтын

#### IDEAS: 

* think about 'io' indexes on nouns

In [123]:
import pympi
from bs4 import BeautifulSoup
import pandas as pd
from collections import Counter
import json
import re 
import pyconll
from os import walk
import itertools

### 1. Creating a conll-U file and adding the POS-tags

In [2]:
files = []
for (dirpath, dirnames, filenames) in walk('abaza_json'):
    files.extend(filenames)
    break

In [3]:
len(files)

25

In [4]:
with open('abaza_json/' + files[0]) as json_file:
        data = json.load(json_file)
        
data

{'meta': {'filename': 'corpus/conlab_abaza/eaf/2018.07.16_BidzhevaTA_1932_retsept.eaf',
  'title': 'Рецепт',
  'speakers': 'tab1932_f',
  'duration': "00'01''",
  'recording_date': '16.07.2018',
  'recording_place': 'Инжич-Чукун',
  'annotators_linguists': 'DA',
  'annotators_consultants': 'unknown'},
 'sentences': [{'text': '\n[tab1932_f] квтIАгъьгIважва гIатурЫчIвитIта',
   'words': [{'off_start': 0,
     'off_end': 1,
     'wf': '\n',
     'wtype': 'punc',
     'next_word': 1},
    {'off_start': 1,
     'off_end': 12,
     'wf': '[tab1932_f]',
     'wtype': 'punc',
     'next_word': 2},
    {'wf': 'квтIагъьгIважва',
     'off_start': 13,
     'off_end': 28,
     'wtype': 'word',
     'ana': [{'lex': "kwtaR'?waza",
       'lex_acute': 'kʷṭáʁ’ʕʷaẑa',
       'gr.pos': 'kʷṭáʁ’-ʕʷa-ẑa',
       'parts': 'квтӏАгъьгӏважва',
       'gloss': 'яйцо-два-десять'}],
     'next_word': 3,
     'sentence_index': 0},
    {'wf': 'гIатурычIвитIта',
     'off_start': 29,
     'off_end': 44,
     '

Opening the json for individual text and parsing it.

In [5]:
tags = []
out = []
i = 1

for file in files:
    texts = []
    trans = []
    
    with open('abaza_json/' + file) as json_file:
        data = json.load(json_file)

    for element in data.get('sentences'): 
        sent = element.get('text')
        words = []

        for word in element.get('words'):
            glosses = word.get('ana')
            if isinstance(glosses, list):
                words.append((glosses[0].get('parts'), glosses[0].get('gloss'), glosses[0].get('parts')))
        if len(words) > 0:
            texts.append([sent.replace('\n', ''), words])
        else:
            trans.append(sent.replace('\n', ''))

    for element, trans in zip(texts, trans):
        try:
            sent_id = i
            text_name = re.search(r'(.+).json', file).group(1)
            text = element[0]
            trans = trans

            string = '# sent_id = {}\n# text_name = {}\n# text = {}\n# ft_rus = {}\n'.format(sent_id, text_name, text, trans)
            out.append(string)

            k = 1
        
            for word in element[1]:
                gloss = re.sub(r'[а-я]|[А-Я]', '', word[1])
                gloss = re.sub(r'-', '|', gloss)
                gloss = re.sub(r'_', '', gloss)
                gloss = re.sub(r'\?', '', gloss)
                gloss = re.sub(r'\(', '', gloss)
                gloss = re.sub(r'\)', '', gloss)
                gloss = re.sub(r'ё', '', gloss)
                gloss = re.sub(r'\[', '', gloss)
                gloss = re.sub(r'\]', '', gloss)
                gloss = re.sub(r'(\|)+', '|', gloss) 
                gloss = re.sub(r'-$|^-', '', gloss)
                gloss = re.sub(r'\|$|^\|', '', gloss)

                pos = '_'

                if len(gloss)<2:
                    gloss = '_'
                    # experimental: try to assign to anything without gloss
                    # pos = 'NOUN'
                elif 'adv' in gloss:
                    pos = 'ADV'
                elif 'abs' in gloss:
                    pos = 'VERB'
                elif 'erg' in gloss:
                    pos = 'VERB'
                elif 'def' in gloss:
                    pos = 'NOUN'
                elif 'indf' in gloss: 
                    pos = 'NOUN'
                elif 'pl' in gloss: 
                    pos = 'NOUN'
                elif 'dcl' in gloss: 
                    pos = 'VERB'
                elif 'ins' in gloss: 
                    pos = 'NOUN'
                elif 'dist' in gloss: 
                    pos = 'PRON'
                elif 'msd' in gloss: 
                    pos = 'VERB'
                elif 'coord' in gloss: 
                    pos = 'CCONJ'
#                 elif ['abs', 'erg'] not in gloss and 'io' in gloss:
#                     pos = 'NOUN'
                else: 
                    pos = '_' 

                string = '{}\t{}\t_\t{}\t_\t{}\t_\t_\t_\tSegment={}|Gloss={}\n'.format(k, word[0], pos, gloss, word[2], word[1])
                out.append(string)
                k += 1

                tags.append(gloss)

            out.append('\n')

            i += 1
        
        except TypeError:
            print([i[1] for i in element[1]])

['Гумлокт', '3m.io-брат-indf', 'Кубиналокт', '3m.io-два-inc-ipf', '3m.io-брат', 'Инджичлокт-add', '3m.io-три-inc-ipf', '3m.io-брат', None]
['рЫцӏа', 'й-глАвн-у', None, None, None, None]
['плохой-adv', 'относиться', 'rel.erg-делать-ipf-pst.nfin rel.abs-3n.io-ben-плохой-pst.nfin', '1pl.io-государство', 'rel.abs-3n.io-ben-плохой-pst', '3sg.erg-говорить-dcl', 'десять-lnk-один-миллион-unit', 'доллар-ins', '3n.abs-csl-1sg.io-3m.erg-давать-ipf-pst', '3n.abs-neg.emp-3m.io-1sg.erg-neg-?-dcl', '3m.erg-говорить-dcl', None]
['1sg.abs-горец-cnc', 'мы', '1pl.io-возле', 'dist-похожий', 'neg.emp-быть-neg', '1sg.erg-сказал-dcl', 'dist.sg', 'rel.mnr-3m.io-dat-1sg.erg-сказать-pst.nfin', 'старуха-indf', 'ребёнок-add', '3f.io-спина', '3h.abs-loc-сидеть-lat-adv', '3h.abs-csl-идти-pst', 'хвать-quot', 'def-палка-шест', 'csl-3f.erg-взять-pst', 'def-filler', 'def.верёвка', '3h.abs-loc-перейти-pst', 'def-палка-add', 'loc-3f.erg-caus-падать-pst', '3h.abs-loc-loc.elat-войти-re-dcl', None]
['dist.pl', '1sg.erg-дела

Conll-u transformation:

In [6]:
data = ''.join(out)

In [7]:
print(data[1:1000])

 sent_id = 1
# text_name = 2018.07.16_BidzhevaTA_1932_retsept
# text = [tab1932_f] квтIАгъьгIважва гIатурЫчIвитIта
# ft_rus = [tab1932_f] Слив двадцать яиц
1	квтӏАгъьгӏважва	_	_	_	_	_	_	_	Segment=квтӏАгъьгӏважва|Gloss=яйцо-два-десять
2	гӏатурЫчӏвитӏта	_	ADV	_	csl|loc|2m.erg|caus|prs|dcl|adv	_	_	_	Segment=гӏатурЫчӏвитӏта|Gloss=csl-loc-2m.erg-caus-литься-prs-dcl-adv

# sent_id = 2
# text_name = 2018.07.16_BidzhevaTA_1932_retsept
# text = ауИ аъарА столОвая лОжка дзЫта йАлаучвахитI
# ft_rus = Туда вливаешь столько же столовых ложек воды.
1	ауи	_	PRON	_	dist.sg	_	_	_	Segment=ауи|Gloss=dist.sg
2	аъарА	_	_	_	_	_	_	_	Segment=аъарА|Gloss=столько
3	столовая	_	_	_	_	_	_	_	Segment=столовая|Gloss=столовая
4	ложка	_	_	_	_	_	_	_	Segment=ложка|Gloss=ложка
5	дзЫ-та	_	ADV	_	adv	_	_	_	Segment=дзЫ-та|Gloss=вода-adv
6	й-А-ла-у-чва-х-и-тӏ	_	VERB	_	3n.abs|3n.io|loc|2m.erg|re|prs|dcl	_	_	_	Segment=й-А-ла-у-чва-х-и-тӏ|Gloss=3n.abs-3n.io-loc-2m.erg-вливать-re-prs-dcl

# sent_id = 3
# text_name = 2018.07.16_Bi


In [16]:
with open("atb_abaza-ud-train.conllu", "w") as text_file:
    text_file.write(data)

### 2. More detailled annotation attempts

Working with tags unique tags:

In [17]:
len(set(tags))

1283

In [18]:
tags_clean = []
for elemnt in tags:
    tags_clean.append(elemnt.split('|'))

merged = list(itertools.chain.from_iterable(tags_clean))
len(set(merged))

224

### 2. Lemmatizing

In [189]:
derivational_morphemes = ['с-', 'б-', 'д-', 'й-', 'хI-', 'x1-', 'хiы-', 'хIа-', 'ры-',
                          'шв-', 'на-', 'а-', 'л-', 'р-', 'з-',
                         'тш-', 'аба-', 'а-', 'чв-', 'з-', 'ц-', 
                          'ма-', 'мхъа-', 'ла-', 'ъа-', 'ан-', 'ш-', 
                          'з-', 'дза-', 'дзы-', 'гь-', 'м-', 'ата-', 'ба-', 
                          'гIа-', 'на-', 'р-', 'ай-', '-х', '-ха',
                          '-ла', '-запыт', '-ркIва', '-зла', '-уа', '-у',
                          '-у', '-з', '-р(а)', '-хьа', '-с', '-тI', '-д', 
                          '-пI', '-б', '-и', '-н', 
                          '-ш', '-ква', '-чва', '-кI', '-ба', '-па', '-гIв', 
                          '-ла', '-та', '-дъа', '-дза', '-и', '-гьи', 
                          '-зтын', '-рквын', '-ркIвын', '-мца', '-ма', '-йа', '-да', 
                          '-зд', '-ргIад', '-нда', '-за', 
                          '-хIва', '-гIв', '-рта', '-тI', '-кI', ]

derivational_morphemes = [i.lower() for i in derivational_morphemes]

len(derivational_morphemes)

86

In [190]:
dev_initial = pyconll.load_from_file('atb_abaza-ud-dev.conllu')
dev_new = pyconll.load_from_file('atb_abaza-ud-train-new.conllu')

Change tokens to russian orthography and add lemmas.

In [193]:
aux_lemmas = set()
for sentence in zip(dev_new, dev_initial):
    for token in zip(sentence[0], sentence[1]):
        token[0]._form = token[0].form.replace('-', '')
        token[0]._form = token[0].form.replace('ӏ', 'I')
        token[0]._form = token[0].form.replace('1', 'I')
        token[1]._form = token[0].form.lower()
        token[1]._form = token[1].form.replace('i', 'I')
        
        for w in token[1].misc['Gloss']:
            
            #plain forms
            if '-' not in w:
                token[1].lemma = token[1]._form
            
            #nouns
            if re.match(r'^[1-3](sg|pl)\.io-|^def-', w):
                if token[1].upos != 'VERB':
                    for f in token[1].misc['Segment']:
                        f = f.replace('ӏ', 'I')
                        f = f.replace('i', 'I')
                        f = f.replace('(', '')
                        f = f.replace(')', '')
                        f = f.lower()
                        new = re.sub('|'.join(derivational_morphemes), '', f)
                        new = new.replace('-', '')
                        token[1].lemma = 'а' + new
                        
            if 

In [194]:
with open("atb_abaza-ud-dev-conv.conllu", "w") as text_file:
    text_file.write(dev_initial.conll())

### 3. Adding propper tags 

In [71]:
!cat atb_abaza-ud-train.conllu | python3 conllu-feats.py abazaXPOS.udx > atb_abaza-ud-train-new.conllu

5.0 {'def'} {'NOUN', 'Definite=Def'}
5.0 {'indef'} {'NOUN', 'Definite=Ind'}
5.0 {'indf'} {'NOUN', 'Definite=Ind'}
5.0 {'ins'} {'Case=Ins'}
5.0 {'dat'} {'Case=Dat'}
5.0 {'neg'} {'Polarity=Neg'}
5.0 {'pl'} {'NOUN', 'Number=Plur'}
5.0 {'pl.h'} {'NOUN', 'Number=Plur'}
5.0 {'unit'} {'Number=Count'}
5.0 {'cln'} {'NUM'}
5.0 {'clh'} {'NUM'}
5.0 {'clh'} {'NUM'}
5.0 {'adv'} {'ADV'}
5.0 {'3h.abs'} {'Number[abs]=Sing', 'Person[abs]=3', 'Gender[abs]=Com'}
5.0 {'1sg.erg'} {'Person[erg]=1', 'Number[erg]=Sing'}
5.0 {'1sg.abs'} {'Number[abs]=Sing', 'Person[abs]=1'}
5.0 {'1sg.io'} {'Number[dat]=Sing', 'Person[dat]=1'}
5.0 {'2m.erg'} {'Person[erg]=2', 'Gender[erg]=Masc', 'Number[erg]=Sing'}
5.0 {'2m.abs'} {'Number[abs]=Sing', 'Gender[abs]=Masc', 'Person[abs]=2'}
5.0 {'2m.io'} {'Number[dat]=Sing', 'Gender[dat]=Masc', 'Person[dat]=2'}
5.0 {'2f.erg'} {'Person[erg]=2', 'Gender[erg]=Fem', 'Number[erg]=Sing'}
5.0 {'2f.abs'} {'Number[abs]=Sing', 'Gender[abs]=Fem', 'Person[abs]=2'}
5.0 {'2f.io'} {'Gender[dat]=Fe

- {'_', 'npst.dcl'} {'npst.dcl'} {'_'} {'VERB', 'Tense=Pres'} ||| _ Number[abs]=Sing|Gender[abs]=Masc|Person[abs]=2 _
> {'_', 'add'}
> {'_', 'npst.dcl', '2m.abs'}
- {'_', 'npst.dcl', '2m.abs'} {'2m.abs'} {'_', 'npst.dcl'} {'Number[abs]=Sing', 'Gender[abs]=Masc', 'Person[abs]=2'} ||| _  _
- {'_', 'npst.dcl'} {'npst.dcl'} {'_'} {'VERB', 'Tense=Pres'} ||| _ Number[abs]=Sing|Gender[abs]=Masc|Person[abs]=2 _
> {'_', 'add'}
> {'_', 'add'}
> {'_', 'npst.dcl', '2m.abs'}
- {'_', 'npst.dcl', '2m.abs'} {'2m.abs'} {'_', 'npst.dcl'} {'Number[abs]=Sing', 'Gender[abs]=Masc', 'Person[abs]=2'} ||| _  _
- {'_', 'npst.dcl'} {'npst.dcl'} {'_'} {'VERB', 'Tense=Pres'} ||| _ Number[abs]=Sing|Gender[abs]=Masc|Person[abs]=2 _
> {'_', 'add'}
> {'_', 'npst.dcl', '2m.abs'}
- {'_', 'npst.dcl', '2m.abs'} {'2m.abs'} {'_', 'npst.dcl'} {'Number[abs]=Sing', 'Gender[abs]=Masc', 'Person[abs]=2'} ||| _  _
- {'_', 'npst.dcl'} {'npst.dcl'} {'_'} {'VERB', 'Tense=Pres'} ||| _ Number[abs]=Sing|Gender[abs]=Masc|Person[abs]=2 _


> {'_', '3pl.io'}
- {'_', '3pl.io'} {'3pl.io'} {'_'} {'Person[dat]=3', 'Number[dat]=Plur'} ||| _  _
> {'_', 'npst.dcl', '3m+cop'}
- {'_', 'npst.dcl', '3m+cop'} {'npst.dcl'} {'_', '3m+cop'} {'VERB', 'Tense=Pres'} ||| _  _
> {'_', '3m.erg', 'dcl'}
- {'_', '3m.erg', 'dcl'} {'3m.erg'} {'_', 'dcl'} {'Gender[erg]=Masc', 'Person[erg]=3', 'Number[erg]=Sing'} ||| _  _
> {'_', '3pl.io'}
- {'_', '3pl.io'} {'3pl.io'} {'_'} {'Person[dat]=3', 'Number[dat]=Plur'} ||| _  _
> {'_', 'pst', '3m+cop'}
- {'_', 'pst', '3m+cop'} {'pst'} {'_', '3m+cop'} {'VERB', 'Tense=Pst'} ||| _  _
> {'_'}
> {'_', '3pl.erg', 'pst.nfin', 'rel.io', 'ben', 'ipf'}
- {'_', '3pl.erg', 'pst.nfin', 'rel.io', 'ben', 'ipf'} {'3pl.erg'} {'_', 'pst.nfin', 'rel.io', 'ben', 'ipf'} {'Number[erg]=Plur', 'Person[erg]=3'} ||| _  _
- {'_', 'pst.nfin', 'rel.io', 'ben', 'ipf'} {'ben'} {'_', 'ipf', 'pst.nfin', 'rel.io'} {'VERB', 'Case=Ben'} ||| _ Number[erg]=Plur|Person[erg]=3 _
- {'_', 'ipf', 'pst.nfin', 'rel.io'} {'ipf'} {'_', 'pst.nfin', 'rel

- {'_', '1sg.io'} {'1sg.io'} {'_'} {'Number[dat]=Sing', 'Person[dat]=1'} ||| _  _
> {'_', '3pl.abs', 'loc', 'dcl'}
- {'_', '3pl.abs', 'loc', 'dcl'} {'3pl.abs'} {'_', 'loc', 'dcl'} {'Number[abs]=Plur', 'Person[abs]=3'} ||| _  _
> {'_'}
> {'_'}
> {'_', 'add'}
> {'_', 'neg', 'neg.emp', '1sg.io', 're'}
- {'_', 'neg', 'neg.emp', '1sg.io', 're'} {'neg'} {'_', 'neg.emp', 're', '1sg.io'} {'Polarity=Neg'} ||| _  _
- {'_', 'neg.emp', 're', '1sg.io'} {'1sg.io'} {'_', 'neg.emp', 're'} {'Number[dat]=Sing', 'Person[dat]=1'} ||| _ Polarity=Neg _
- {'_', 'neg.emp', 're'} {'re'} {'_', 'neg.emp'} {'VERB', 'Aspect=Refactive'} ||| _ Polarity=Neg|Number[dat]=Sing|Person[dat]=1 _
> {'_', '1sg.abs', 'dcl', 'csl', 're'}
- {'_', '1sg.abs', 'dcl', 'csl', 're'} {'1sg.abs'} {'_', 'csl', 're', 'dcl'} {'Number[abs]=Sing', 'Person[abs]=1'} ||| _  _
- {'_', 'csl', 're', 'dcl'} {'re'} {'_', 'csl', 'dcl'} {'VERB', 'Aspect=Refactive'} ||| _ Number[abs]=Sing|Person[abs]=1 _
> {'_', 'pl', 'def'}
- {'_', 'pl', 'def'} {'def

- {'_', 'caus', 'dcl'} {'caus'} {'_', 'dcl'} {'VERB', 'Voice=Cau'} ||| _ Gender[erg]=Masc|Person[erg]=3|Number[erg]=Sing|Number[abs]=Sing|Gender[abs]=Neut|Person[abs]=3 _
> {'_', 'loc', 'neg.emp', 'neg', 'dcl', 'nondum', 'csl', '3n.abs'}
- {'_', 'loc', 'neg.emp', 'neg', 'dcl', 'nondum', 'csl', '3n.abs'} {'neg'} {'_', 'loc', 'neg.emp', 'dcl', 'nondum', 'csl', '3n.abs'} {'Polarity=Neg'} ||| _  _
- {'_', 'loc', 'neg.emp', 'dcl', 'nondum', 'csl', '3n.abs'} {'3n.abs'} {'_', 'loc', 'neg.emp', 'dcl', 'nondum', 'csl'} {'Number[abs]=Sing', 'Gender[abs]=Neut', 'Person[abs]=3'} ||| _ Polarity=Neg _
> {'_', 'dist.sg'}
> {'_', '3n.io', 'adv'}
- {'_', '3n.io', 'adv'} {'adv'} {'_', '3n.io'} {'ADV'} ||| _  _
- {'_', '3n.io'} {'3n.io'} {'_'} {'Person[dat]=3', 'Number[dat]=Sing', 'Gender[dat]=Neut'} ||| ADV  _
> {'_'}
> {'_'}
> {'_', 'neg', 'neg.emp', 'dcl', 'csl', '3h.abs'}
- {'_', 'neg', 'neg.emp', 'dcl', 'csl', '3h.abs'} {'neg'} {'_', 'neg.emp', 'dcl', 'csl', '3h.abs'} {'Polarity=Neg'} ||| _  _
- {'_